Полезная страница
http://statsmodels.sourceforge.net/devel/stats.html

### Интервальная оценка среднего

In [2]:
from statsmodels.stats.weightstats import _zconfint_generic, _tconfint_generic, zconfint
from scipy import stats

In [ ]:
print('95%% confidence interval for the mean time: [%f, %f]' % zconfint(sample))

In [ ]:
_zconfint_generic(Выборочное среднее, стандартная ошибка среднего(Станд. откл./Корень из объема выборки), Альфа(0.05), 'two-sided'))

In [ ]:
_tconfint_generic(Выборочное среднее, стандартная ошибка среднего, Кол-во степеней свободы (n - 1), 0.05, 'two-sided'))

In [ ]:
zconfint(data.Placebo) - интервальная оценка среднего по выборке

In [ ]:
t пригодится тогда, когда мы изначально не знаем, какая дисперсия в генеральной совокупности. z можно применять для выборок с N>30, потому что ЦПТ порождает нормальное распределение выборочного среднего.

### Доверительный интервал для доли

In [1]:
from statsmodels.stats.proportion import proportion_confint

In [ ]:
normal_interval = proportion_confint(sum(random_sample), len(random_sample), method = 'normal')

In [ ]:
wilson_interval = proportion_confint(sum(random_sample), len(random_sample), method = 'wilson')

In [ ]:
Интервал Уилсона лучше. Особенно когда случайная подвыборка мала. 

In [ ]:
Проверка гипотезы о равенстве долей: хи-квадрат:
stats.chi2_contingency(matrix)

### Размер выборки для интервала заданной ширины

In [5]:
from statsmodels.stats.proportion import samplesize_confint_proportion

In [ ]:
n_samples = int(np.ceil(samplesize_confint_proportion(sample.mean(), половина нужной нам ширины интервала)))

### Доверительные интервалы для разности долей

In [39]:
import scipy
from statsmodels.stats.proportion import proportion_confint

In [ ]:
def proportions_confint_diff_ind(sample1, sample2, alpha = 0.05):    
    z = scipy.stats.norm.ppf(1 - alpha / 2.)   
    p1 = float(sum(sample1)) / len(sample1)
    p2 = float(sum(sample2)) / len(sample2)
    
    left_boundary = (p1 - p2) - z * np.sqrt(p1 * (1 - p1)/ len(sample1) + p2 * (1 - p2)/ len(sample2))
    right_boundary = (p1 - p2) + z * np.sqrt(p1 * (1 - p1)/ len(sample1) + p2 * (1 - p2)/ len(sample2))
    
    return (left_boundary, right_boundary)

In [ ]:
Доверительные интервалы для разности долей (связанные выборки)

In [ ]:
def proportions_confint_diff_rel(sample1, sample2, alpha = 0.05):
    z = scipy.stats.norm.ppf(1 - alpha / 2.)
    sample = list(zip(sample1, sample2))
    n = len(sample)
        
    f = sum([1 if (x[0] == 1 and x[1] == 0) else 0 for x in sample])
    g = sum([1 if (x[0] == 0 and x[1] == 1) else 0 for x in sample])
    
    left_boundary = float(f - g) / n  - z * np.sqrt(float((f + g)) / n**2 - float((f - g)**2) / n**3)
    right_boundary = float(f - g) / n  + z * np.sqrt(float((f + g)) / n**2 - float((f - g)**2) / n**3)
    return (left_boundary, right_boundary)

### Доверительные интервалы на основе бутстрепа

In [41]:
import numpy as np
import pandas as pd

In [ ]:
def get_bootstrap_samples(data, n_samples):
    indices = np.random.randint(0, len(data), (n_samples, len(data)))
    samples = data[indices]
    return samples
    
Есть вариант в виде итератора, чтобы не держать в памяти огромный массив
def get_bootstrap_samples(data, n_samples):
    i = 0
    while i < n_samples:
        indices = np.random.randint(0, len(data), len(data))
        sample = data[indices]
        yield sample
        i += 1

In [ ]:
def stat_intervals(stat, alpha):
    boundaries = np.percentile(stat, [100 * alpha / 2., 100 * (1 - alpha / 2.)])
    return boundaries

In [ ]:
#Интервальная оценка медианы
sample_median_scores = list(map(np.median, get_bootstrap_samples(sample, 1000)))
stat_intervals(sample_median_scores, 0.05)

In [ ]:
#Интервальная оценка разности медиан
delta_median_scores = list(map(lambda x: x[1] - x[0], zip(sample1_median_scores, sample2_median_scores)))
stat_intervals(delta_median_scores, 0.05)

### Одновыборочный критерий Стьюдента

In [42]:
import scipy
from statsmodels.stats.weightstats import *

In [ ]:
stats.ttest_1samp(data.Placebo, 50.0)

In [ ]:
zconfint(data.Placebo) - интервальная оценка среднего по выборке

### Проверка на нормальность

In [ ]:
QQ-плот

In [ ]:
plt.figure(figsize=(12,8))
plt.subplot(2,2,1)
stats.probplot(lech.ix[:, 1], dist="norm", plot=plt)
plt.subplot(2,2,2)
stats.probplot(ped.ix[:, 1], dist="norm", plot=plt)
plt.show()

In [ ]:
Критерий Шапиро - Уилка

In [ ]:
stats.shapiro(data.Placebo)

### Двухвыборочный критерий Стьюдента

Тут надо помнить, что генеральные дисперсии должны быть одинаковы (проблема Беренса-Фишера). 
Но мы, как правило, не обладаем подобной инфой, так что над прописывать equal_var=False, чтобы использовать t-критерий Уэлша (и вообще лучше всегда его юзать). 
Если так уж надо будет, сравнивать дисперсии хорошо через scipy.stats.levene

In [ ]:
scipy.stats.ttest_ind(data.Placebo, data.Methylphenidate, equal_var = False)

In [ ]:
cm = CompareMeans(DescrStatsW(data.Methylphenidate), DescrStatsW(data.Placebo))
cm.tconfint_diff(usevar='unequal')

In [ ]:
#И для связанных выборок
stats.ttest_rel(data.Methylphenidate, data.Placebo)

In [ ]:
DescrStatsW(data.Methylphenidate - data.Placebo).tconfint_mean() - доверительный интервал разности для связанных выборок

### Z-критерий для разности долей

In [ ]:
def proportions_diff_z_stat_ind(sample1, sample2):
    n1 = len(sample1)
    n2 = len(sample2)
    
    p1 = float(sum(sample1)) / n1
    p2 = float(sum(sample2)) / n2 
    P = float(p1*n1 + p2*n2) / (n1 + n2)
    
    return (p1 - p2) / np.sqrt(P * (1 - P) * (1. / n1 + 1. / n2))

In [ ]:
def proportions_diff_z_test(z_stat, alternative = 'two-sided'):
    if alternative not in ('two-sided', 'less', 'greater'):
        raise ValueError("alternative not recognized\n"
                         "should be 'two-sided', 'less' or 'greater'")
    
    if alternative == 'two-sided':
        return 2 * (1 - scipy.stats.norm.cdf(np.abs(z_stat)))
    
    if alternative == 'less':
        return scipy.stats.norm.cdf(z_stat)

    if alternative == 'greater':
        return 1 - scipy.stats.norm.cdf(z_stat)

In [ ]:
proportions_diff_z_test(proportions_diff_z_stat_ind(sample1, sample2))

Для связанных выборок:

In [ ]:
def proportions_diff_z_stat_rel(sample1, sample2):
    sample = list(zip(sample1, sample2))
    n = len(sample)
    
    f = sum([1 if (x[0] == 1 and x[1] == 0) else 0 for x in sample])
    g = sum([1 if (x[0] == 0 and x[1] == 1) else 0 for x in sample])
    
    return float(f - g) / np.sqrt(f + g - float((f - g)**2) / n )
    
#Реализация расчета p-value двумя ячейками выше. 

In [ ]:
 proportions_diff_z_test(proportions_diff_z_stat_rel(data.banner_a, data.banner_b), 'less'))

### Одновыборочный критерий знаков

In [ ]:
print("M: %d, p-value: %f" % sign_test(sample, медиана))

### Одновыборочный критерий знаковых рангов Вилкоксона

In [ ]:
m0 = 0.5
stats.wilcoxon(sample - m0)

### Одновыборочный перестановочный критерий

In [ ]:
#Считаем t-статистику
def permutation_t_stat_1sample(sample, mean):
    t_stat = sum(list(map(lambda x: x - mean, sample)))
    return t_stat

In [ ]:
#Генерация выборок с возможными перестановками знаков, дабы восстановить распределение
def permutation_zero_distr_1sample(sample, mean, max_permutations = None):
    centered_sample = list(map(lambda x: x - mean, sample))
    if max_permutations:
        signs_array = set([tuple(x) for x in 2 * np.random.randint(2, size = (max_permutations, 
                                                                              len(sample))) - 1 ])
    else:
        signs_array =  itertools.product([-1, 1], repeat = len(sample))
    distr = [sum(centered_sample * np.array(signs)) for signs in signs_array]
    return distr

In [ ]:
pylab.hist(permutation_zero_distr_1sample(mouses_data.proportion_of_time, (среднее, с которым сравнить), bins = 15)
pylab.show()

In [ ]:
Считаем p-value
def permutation_test(sample, mean, max_permutations = None, alternative = 'two-sided'):
    if alternative not in ('two-sided', 'less', 'greater'):
        raise ValueError("alternative not recognized\n"
                         "should be 'two-sided', 'less' or 'greater'")
    
    t_stat = permutation_t_stat_1sample(sample, mean)
    
    zero_distr = permutation_zero_distr_1sample(sample, mean, max_permutations)
    
    if alternative == 'two-sided':
        return sum([1. if abs(x) >= abs(t_stat) else 0. for x in zero_distr]) / len(zero_distr)
    
    if alternative == 'less':
        return sum([1. if x <= t_stat else 0. for x in zero_distr]) / len(zero_distr)

    if alternative == 'greater':
        return sum([1. if x >= t_stat else 0. for x in zero_distr]) / len(zero_distr)

In [ ]:
print("p-value: %f" % permutation_test(выборка, среднее, количество перестановок(например,10000)))

### Двухвыборочный критерий знаков для связанных выборок

In [ ]:
sign_test(sample1 - sample2)

### Двухвыборочный критерий знаковых рангов Вилкоксона для связанных выборок

In [ ]:
stats.wilcoxon(sample1, sample2)

### Двухвыборочный перестановочный критерий для связанных выборок

In [ ]:
Все то же самое, что мы делали с одновыборочным. Вычисляем t-статистику
def permutation_t_stat_1sample(sample, mean):
    t_stat = sum(map(lambda x: x - mean, sample))
    return t_stat

In [ ]:
Считаем нулевое распределение
def permutation_zero_distr_1sample(sample, mean, max_permutations = None):
    centered_sample = list(map(lambda x: x - mean, sample))
    if max_permutations:
        signs_array = set([tuple(x) for x in 2 * np.random.randint(2, size = (max_permutations, 
                                                                              len(sample))) - 1 ])
    else:
        signs_array =  itertools.product([-1, 1], repeat = len(sample))
    distr = [sum(centered_sample * np.array(signs)) for signs in signs_array]
    return distr

In [ ]:
Можно сразу посмотреть на нулевое распределение
pylab.hist(permutation_zero_distr_1sample(weight_data.After - weight_data.Before, 0., 
                               max_permutations = 10000))
pylab.show()

In [ ]:
Ну и считаем p-value
def permutation_test(sample, mean, max_permutations = None, alternative = 'two-sided'):
    if alternative not in ('two-sided', 'less', 'greater'):
        raise ValueError("alternative not recognized\n"
                         "should be 'two-sided', 'less' or 'greater'")
    
    t_stat = permutation_t_stat_1sample(sample, mean)
    
    zero_distr = permutation_zero_distr_1sample(sample, mean, max_permutations)
    
    if alternative == 'two-sided':
        return sum([1. if abs(x) >= abs(t_stat) else 0. for x in zero_distr]) / len(zero_distr)
    
    if alternative == 'less':
        return sum([1. if x <= t_stat else 0. for x in zero_distr]) / len(zero_distr)

    if alternative == 'greater':
        return sum([1. if x >= t_stat else 0. for x in zero_distr]) / len(zero_distr)

In [ ]:
print("p-value: %f" % permutation_test(weight_data.After - weight_data.Before, 0., 
                               max_permutations = 1000))

В случае критерия знаков под средним мы понимаем медиану. В случае перестановочного -- матожидание разности. Знаковые критерии используют только знаки перед значениями, не учитывает абсолютные значения. Ранговые критерии используют порядки. Перестановочный критерий использует гораздо больше информации, поскольку учитывает значения. 

### Ранговый критерий Манна-Уитни для независимых выборок

In [ ]:
stats.mannwhitneyu(sample1, sample2)

### Перестановочный критерий для независмых выборок

In [ ]:
def permutation_t_stat_ind(sample1, sample2):
    return np.mean(sample1) - np.mean(sample2)

In [ ]:
def get_random_combinations(n1, n2, max_combinations):
    index = list(range(n1 + n2))
    indices = set([tuple(index)])
    for i in range(max_combinations - 1):
        np.random.shuffle(index)
        indices.add(tuple(index))
    return [(index[:n1], index[n1:]) for index in indices]

In [ ]:
def permutation_zero_dist_ind(sample1, sample2, max_combinations = None):
    joined_sample = np.hstack((sample1, sample2))
    n1 = len(sample1)
    n = len(joined_sample)
    
    if max_combinations:
        indices = get_random_combinations(n1, len(sample2), max_combinations)
    else:
        indices = [(list(index), filter(lambda i: i not in index, range(n))) \
                    for index in itertools.combinations(range(n), n1)]
    
    distr = [joined_sample[list(i[0])].mean() - joined_sample[list(i[1])].mean() \
             for i in indices]
    return distr

In [ ]:
pylab.hist(permutation_zero_dist_ind(price2001, price2002, max_combinations = 1000))
pylab.show()

In [ ]:
def permutation_test(sample, mean, max_permutations = None, alternative = 'two-sided'):
    if alternative not in ('two-sided', 'less', 'greater'):
        raise ValueError("alternative not recognized\n"
                         "should be 'two-sided', 'less' or 'greater'")
    
    t_stat = permutation_t_stat_ind(sample, mean)
    
    zero_distr = permutation_zero_dist_ind(sample, mean, max_permutations)
    
    if alternative == 'two-sided':
        return sum([1. if abs(x) >= abs(t_stat) else 0. for x in zero_distr]) / len(zero_distr)
    
    if alternative == 'less':
        return sum([1. if x <= t_stat else 0. for x in zero_distr]) / len(zero_distr)

    if alternative == 'greater':
        return sum([1. if x >= t_stat else 0. for x in zero_distr]) / len(zero_distr)

In [ ]:
print("p-value: %f" % permutation_test(price2001, price2002, max_permutations = 10000))

### Корреляция

In [ ]:
DataFrame.corr(method='spearman')

Можно использовать это:
from scipy.stats import pearsonr

Критерии для проверки значимости корреляции: 
- Критерий Стьюдента для корреляции Пирсона и Спирмена
- Критерий хи-квадрат для корреляции Мэтьюса и коэффициента V Крамера

In [ ]:
Корреляция Мэтьюса:
def matthews_coeff(a, b, c, d):
    """Calculates Matthew's correlation coefficient of a given contingency table [a, b], [c, d]
    (https://en.wikipedia.org/wiki/Matthews_correlation_coefficient).

        Parameters
        ----------
        a : integer, negative value count of the first binary feature.
        b : integer, positive value count of the first binary feature.
        c : integer, negative value count of the second binary feature.
        d : integer, positive value count of the second binary feature.

        Returns
        -------
        coeff : float, -1 <= coeff <= 1.
        """
    assert all([a, b, c, d]) >= 0, 'arguments can not be less than zero'
    return float(a * d - b * c) / np.sqrt((a + b) * (a + c) * (b + d) * (c + d))

In [ ]:
Коэффициент Крамера:
def cramers_coeff(table):
    """Calculates Cramer's coefficient of a given contingency table (https://en.wikipedia.org/wiki/Cram%C3%A9r%27s_V).

        Parameters
        ----------
        table : array-like, displays the (multivariate) frequency distribution of the variables.

        Returns
        -------
        coeff : float, 0 <= coeff <= 1.
        """
    chi2 = sts.chi2_contingency(table)[0]
    n = table.sum()
    return np.sqrt(chi2 / (n*(min(table.shape)-1)))

### Поправка на множественную проверку

In [ ]:
from statsmodels.sandbox.stats.multicomp import multipletests 

In [ ]:
Метод Холма:
reject, p_corrected, a1, a2 = multipletests(sales_correlation.p, 
                                            alpha = 0.05, 
                                            method = 'holm') 

In [ ]:
Метод Бенджамини - Хохберга:
reject, p_corrected, a1, a2 = multipletests(sales_correlation.p, 
                                            alpha = 0.05, 
                                            method = 'fdr_bh') 

Стало быть, мы передаем список p-значений, нужный нам уровень значимости и метод. 
Получаем ответ, отклонить ли нам нулевую гипотезу, а также скоррективрованное значение р. 